## KRWordRank: method for word / keyword extraction

KRWordRank는 Kim et al.(2014)^[1]의 논문을 바탕으로 한 비지도학습기반 단어 추출 기법으로, 데이터기반으로 주요단어 (키워드)를 추출하는 알고리즘이다. 하나의 도메인에 대한 문서들을 바탕으로 명사/형용사/동사/부사 (L set) 중에서 빈도수가 높거나, 주요 단어들과 함께 등장하는 단어를 키워드로 추출한다. KRWordRank는 이름에서 나타나는바와 같이 단어 후보 (subtokens)을 이용하여 word-graph를 생성한 뒤, PageRank의 랭킹 학습 방식을 이용하여 word-graph의 hub subtokens을 추출한다. 

KRWordRank는 다음의 가정을 기반으로 단어를 추출한다. **단어 주변에는 단어가 등장하며, 올바른 단어는 주위의 많은 단어들과 연결되어 있다. 그렇기 때문에 단어는 주위 단어들에 의하여 단어 점수가 보강(reinforced)된다.**


![kr_wordrank_structure](figs/kr_wordrank_fig1.png)


한국어는 의미를 지니는 단어 집합과 문법 기능을 하는 복합형태소 집합으로 나뉘어지며, [문법/명사] + [을/조사]와 같이 어절의 왼쪽에 의미를 지니는 단어인 명사/형용사/동사가 위치한다. 부사는 그 자체로 한 어절을 이룬다. 그렇기 때문에 KRWordRank는 의미있는 단어로서 어절 자체나 어절의 왼쪽에 등장하는 L set을 추출한다. 또한 한국어는 한 글자에 지나치게 많은 의미가 담겨져 있어 해석이 모호하기 때문에 1음절 단어는 추출되는 단어에서 제외한다. 실제로 subtokens으로 이뤄진 word-graph에서 1음절 단어들은 매우 높은 랭킹을 지닌다. KRWordRank는 아래 그림과 같이 subtokens을 어절의 위치에 따라 L/R tags를 부여하여 word-graph를 만든 뒤, 랭킹을 계산한다. 

![kr_wordrank_structure](figs/kr_wordrank_fig2.png)

논문에서 기술되지 않은 후처리(post-processing)가 추가되었다. 영화리뷰의 경우, '영화', '영화가', '영화를' 와 같이 "단어 + R set"이 함께 키워드로 추출된다. 이는 KRWordRank가 주요 L set 혹은 어절을 추출하기 때문이며, '영화', '영화가' 주변 모두 올바른 단어가 위치하기 때문이다. 그렇기 때문에 '영화'라는 단어가 '영화가', '영화를' 등보다 높은 랭킹을 지녔다면, '영화' + R set는 L + R 복합어라 판단하여 제외하였다. 

        keywords = self._select_keywords(lset, rset)

두번째 후처리로, '영화', '음악', '영화음악'이 키워드로 추출되었고, '영화', '음악'이 모두 '영화음악'보다 랭킹이 높을 경우, '영화음악'은 합성어로 판단하여 이를 제거하였다. 

        keywords = self._filter_compounds(keywords)

마지막 후처리로, '스토리'가 상위 랭킹이 될 경우, 한 글자가 랭킹이 높아서 '스토' 역시 키워드로 추출될 수 있다. '스토리'가 상위 랭킹이 된다면 '스토'와 같은 substring은 키워드에서 제거하였다. 

        keywords = self._filter_subtokens(keywords)

사용법은 아래의 예제 코드와 같다. 

[1] Kim, H. J., Cho, S., & Kang, P. (2014). KR-WordRank: An Unsupervised Korean Word Extraction Method Based on WordRank. Journal of Korean Institute of Industrial Engineers, 40(1), 18-33.

In [1]:
def get_texts_scores(fname):
    with open(fname, encoding='utf-8') as f:
        docs = [doc.lower().replace('\n','').split('\t') for doc in f]
        docs = [doc for doc in docs if len(doc) == 2]

        if not docs:
            return [], []

        texts, scores = zip(*docs)
        return list(texts), list(scores)

# La La Land
fname = '../data/134963.txt'
texts, scores = get_texts_scores(fname)

In [2]:
import sys
sys.path.append('../')
from krwordrank.word import KRWordRank
from krwordrank.hangle import normalize
import krwordrank
print(krwordrank.__version__)

1.0.1


단어 추출에 영어/숫자를 포함할 예정이라면 normalize함수를 이용하여 텍스트를 normalize할 것

In [3]:
with open('../data/134963_norm.txt', 'w', encoding='utf-8') as f:
    for text, score in zip(texts, scores):
        text = normalize(text, english=True, number=True)
        f.write('%s\t%s\n' % (text, str(score)))

In [4]:
# La La Land
fname = '../data/134963_norm.txt'
texts, scores = get_texts_scores(fname)

In [5]:
wordrank_extractor = KRWordRank(
    min_count = 5, # 단어의 최소 출현 빈도수 (그래프 생성 시)
    max_length = 10, # 단어의 최대 길이
    verbose = True
    )

beta = 0.85    # PageRank의 decaying factor beta
max_iter = 10

keywords, rank, graph = wordrank_extractor.extract(texts, beta, max_iter)

scan vocabs ... 
num vocabs = 13822
done = 9 Early stopped.


위와 같이 vocabulary를 미리 설정하거나 decaying factor를 단어별로 다르게 (bias) 할당할 수 있으며, 모든 단어의 랭킹의 총 합은 vocabulary size와 같음. 즉 default decaying factor는 1.0

In [6]:
for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:30]:
    print('%8s:\t%.4f' % (word, r))

      영화:	201.9608
      너무:	81.7410
      정말:	40.8016
      음악:	40.3295
     마지막:	38.9302
     뮤지컬:	23.1365
      최고:	22.0345
      사랑:	20.5910
      영상:	20.4099
      아름:	20.3295
      꿈을:	20.3000
     여운이:	19.4569
      진짜:	19.2820
      노래:	18.7877
      보고:	18.4819
      좋았:	17.6783
      그냥:	16.6981
     스토리:	16.2643
      좋은:	15.6323
      인생:	15.4943
      현실:	15.1858
      생각:	14.8838
      지루:	13.7707
      감동:	13.7404
      다시:	13.5948
      보는:	12.4019
      재밌:	11.9944
      좋아:	11.9582
      재미:	11.4880
      좋고:	11.3686


세 가지 영화의 키워드를 비교해보겠습니다. '라라랜드 (134963.txt)', '신세계 (91031.txt)', '엑스맨 (99714.txt)'에 대하여 동일한 방식으로 normalize를 한 뒤, 상위 100개의 키워드들을 비교해보겠습니다.

In [7]:
fnames = ['../data/91031.txt',
          '../data/99714.txt']

for fname in fnames:
    texts, scores = get_texts_scores(fname)
    with open(fname.replace('.txt', '_norm.txt'), 'w', encoding='utf-8') as f:
        for text, score in zip(texts, scores):
            text = normalize(text, english=True, number=True)
            f.write('%s\t%s\n' % (text, str(score)))

In [8]:
top_keywords = []
fnames = ['../data/134963_norm.txt',
          '../data/91031_norm.txt',
          '../data/99714_norm.txt']

for fname in fnames:
    
    texts, scores = get_texts_scores(fname)
    
    wordrank_extractor = KRWordRank(
        min_count=5, max_length=10, verbose=False)
    
    keywords, rank, graph = wordrank_extractor.extract(
        texts, beta, max_iter)
    
    top_keywords.append(
        sorted(keywords.items(),
               key=lambda x:x[1],
               reverse=True)[:100]
    )

In [9]:
movie_names = ['라라랜드', '신세계', '엑스맨']
for k in range(100):
    
    message = '  --  '.join(
        ['%8s (%.3f)' % (top_keywords[i][k][0],top_keywords[i][k][1])
         for i in range(3)])
    
    print(message)
    

      영화 (201.961)  --        영화 (145.840)  --       엑스맨 (112.845)
      너무 (81.741)  --       황정민 (99.563)  --        영화 (65.482)
      정말 (40.802)  --        연기 (89.988)  --        정말 (42.081)
      음악 (40.329)  --        정말 (74.653)  --        진짜 (40.935)
     마지막 (38.930)  --        진짜 (64.803)  --       시리즈 (40.835)
     뮤지컬 (23.137)  --        최고 (54.792)  --        너무 (38.929)
      최고 (22.034)  --        너무 (51.963)  --        재밌 (34.045)
      사랑 (20.591)  --       이정재 (46.503)  --        재미 (30.454)
      영상 (20.410)  --       무간도 (36.144)  --        최고 (28.824)
      아름 (20.329)  --       배우들 (34.579)  --        기대 (24.799)
      꿈을 (20.300)  --        재밌 (28.948)  --       스토리 (24.206)
     여운이 (19.457)  --       스토리 (26.741)  --        역시 (23.917)
      진짜 (19.282)  --        한국 (26.367)  --        보고 (19.043)
      노래 (18.788)  --       신세계 (24.480)  --        액션 (17.983)
      보고 (18.482)  --        대박 (23.847)  --        그냥 (17.412)
      좋았 (17.678)  --       최민식 (19.8

셋 모두 영화이기 때문에 공통된 키워드가 많습니다. top 100에서 중복되는 키워드들을 제거하고 차이가 있는 키워드만 추출해서 살펴보겠습니다. 

In [10]:
keyword_counter = {}
for keywords in top_keywords:
    words, ranks = zip(*keywords)
    for word in words:
        keyword_counter[word] = keyword_counter.get(word, 0) + 1

common_keywords = {word for word, count in keyword_counter.items() if count == 3}
len(common_keywords)

43

세 영화 모두에 등장하는 키워드는 총 43개가 있으며, '스토리', '많이', '진짜' 같은 단어들입니다. 이런 단어를 제외한 selected_top_keywords 리스트를 만든 다음 출력을 해보겠습니다. 

In [11]:
str(common_keywords)

"{'내용', '가장', '이런', '보는', '너무', '좋았', '느낌', '영화', '좋아', '생각', '하나', '스토리', '평점', '내가', '조금', '추천', '그래도', '처음', '있는', '최고', '정말', '사람', '그리고', '봤는데', '재밌', '보면', '이렇게', '하지만', '별로', '모두', '재미', '지루', '보고', '봤습니다', '한번', '진짜', '다시', '기대', '아주', '아니', '마지막', '그냥', '많이'}"

In [12]:
selected_top_keywords = []
for keywords in top_keywords:
    selected_keywords = []
    for word, r in keywords:
        if word in common_keywords:
            continue
        selected_keywords.append((word, r))
    selected_top_keywords.append(selected_keywords)

In [13]:
def get_from_list(l, i, default=('', 0)):
    if len(l) <= i:
        return default
    else:
        return l[i]

라라랜드는 [음악, 사랑, 뮤지컬, 꿈]과 같은 단어들이 나오며, 신세계에서는 [황정민, 이정재, 최민식]과 같은 배우들의 이름과, 홍콩영화 무간도와 주제가 비슷하기에 '무간도'라는 단어, 그리고 ['조폭', '느와르', 잔인'] 같은 영화 분위기와 관련된 내용들이 나옵니다. 또한 '반전'이란 단어에서 반전이 있는 영화라는 것도 알 수 있겠네요. 그에 비하여 엑스맨에서는 캐릭터 이름인 ['울버린', '퀵실버'] 같은 단어들도 나옵니다. ['꿀잼', '마블']과 같은 단어로부터 마블 코믹스의 오락 영화라는 것도 알 수 있습니다. 

In [14]:
for k in range(100 - len(common_keywords) ):
    
    message = '  --  '.join(
        ['%8s (%.3f)' % get_from_list(selected_top_keywords[i], k) for i in range(3)])
    
    print(message)
    

      음악 (40.329)  --       황정민 (99.563)  --       엑스맨 (112.845)
     뮤지컬 (23.137)  --        연기 (89.988)  --       시리즈 (40.835)
      사랑 (20.591)  --       이정재 (46.503)  --        역시 (23.917)
      영상 (20.410)  --       무간도 (36.144)  --        액션 (17.983)
      아름 (20.329)  --       배우들 (34.579)  --      브라이언 (16.782)
      꿈을 (20.300)  --        한국 (26.367)  --       퍼스트 (15.316)
     여운이 (19.457)  --       신세계 (24.480)  --        진심 (14.544)
      노래 (18.788)  --        대박 (23.847)  --        싱어 (14.349)
      좋은 (15.632)  --       최민식 (19.863)  --        완전 (13.027)
      인생 (15.494)  --       느와르 (19.465)  --        명작 (12.585)
      현실 (15.186)  --        완전 (16.765)  --        제일 (10.684)
      감동 (13.740)  --        잔인 (14.026)  --        이건 (10.040)
      좋고 (11.369)  --        역시 (13.526)  --       울버린 (9.162)
      계속 (11.091)  --        특히 (12.954)  --        이번 (9.148)
      결말 (10.639)  --        말이 (12.931)  --       퀵실버 (8.944)
      연기 (10.473)  --        조폭 (12.852)  